 # **Dataset curation**

## Training set

#### From PDB manually download IDs of selected structures, according to the following  advanced search criteria:
- Kunitz domain annotation: PFAM code PF00014
- Resolution method: X-RAY diffraction
- PDB structure resolution: =< 3
- Absence of mutations in the polymer entity

https://www.rcsb.org/search/advanced

Format IDs file to show a single ID for each row 

In [ ]:
# Format to have an ID for each line 
!cat PDB_IDs.txt |tr ',' '\n' > PDB_IDs_rows.txt


# Check output
!head -n 5 PDB_IDs_rows.txt

#### From PDBefold, manually download a summary of pairwise structural alignments of the seed structure with PDB code: 3TGI, chain I, against the whole PDB database.

(3TGI: WILD-TYPE RAT ANIONIC TRYPSIN COMPLEXED WITH BOVINE PANCREATIC TRYPSIN INHIBITOR)

https://www.ebi.ac.uk/msd-srv/ssm/

Credits: E. Krissinel and K. Henrick (2004). Secondary-structure matching (SSM), a new tool for fast protein structure alignment in three dimensions. Acta Cryst. D60, 2256---2268

##### PDB-efold summary filtering for Z-score > 3 and RSMD < 1.5, and formatting to show only ID:chain

In [ ]:
# Filter and format
!awk '{if ($4>=4 && $5<=1.5) print toupper($(NF))}' efold_IDs.txt > efold_IDs_chains.txt

# Check output 
!head -n 5 efold_IDs_chains.txt
# Compare number of entries
!wc efold_IDs.txt
!wc efold_IDs_chains.txt


3TGI:I
1F7Z:I
1FY8:I
1YKT:B
3TGK:I
  657 11763 90840 /content/drive/MyDrive/Bioinformatics/Lab_bioinfomatics_1_2/efold_IDs.txt
 336  336 2352 /content/drive/MyDrive/Bioinformatics/Lab_bioinfomatics_1_2/efold_IDs_chains.txt


#### Merging datasets from different databases

In [ ]:
# Format eFold .txt file with ID:chain rows, to show only IDs
!awk '{print substr($0,1,length($0)-2)}' efold_IDs_chains.txt > efold_IDs_only.txt

# Check output 
!head -n 5 efold_IDs_only.txt

3TGI
1F7Z
1FY8
1YKT
3TGK


In [ ]:
# Merge PDB IDs and PDBeFold IDs in a single file

!comm -12 <(sort efold_IDs_only.txt) <(sort PDB_IDs_rows.txt) > merged_IDs.txt

!wc merged_IDs.txt

119 119 595 /content/drive/MyDrive/Bioinformatics/Lab_bioinfomatics_1_2/merged_IDs.txt


In [ ]:
# Generate a merged file with ID:chain rows from the merged IDs 
# and PDB-eFold file IDs:chains 

# Remove the colon to join
!tr ':' ' ' <efold_IDs_chains.txt > efold_IDs_chains_nocolon.txt

# Merge
!join -j 1 <(sort merged_IDs.txt) <(sort efold_IDs_chains_nocolon.txt) > merged_IDs_chains.txt

# Check output
!cat merged_IDs_chains.txt |sort|uniq|wc

    205     410    1435


#### Retrieve fasta sequences according to the IDs and chains from the merged list

In [ ]:
# Retrieve sequences from PDB in fasta format
!wget https://ftp.rcsb.org/pub/pdb/derived_data/pdb_seqres.txt > pdb.seqres.txt

--2022-05-08 16:26:00--  https://ftp.rcsb.org/pub/pdb/derived_data/pdb_seqres.txt
Resolving ftp.rcsb.org (ftp.rcsb.org)... 132.249.210.142
Connecting to ftp.rcsb.org (ftp.rcsb.org)|132.249.210.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 232080375 (221M) [text/plain]
Saving to: ‘pdb_seqres.txt.1’

pdb_seqres.txt.1    100%[===================>] 221.33M  18.9MB/s    in 13s     

2022-05-08 16:26:14 (17.3 MB/s) - ‘pdb_seqres.txt.1’ saved [232080375/232080375]



In [ ]:
# Formatting for the next command
!awk '{print tolower($1)"_"$2}' merged_IDs_chains.txt > final_merged.txt

# Filtering 
! for i in `cat final_merged.txt` ; do grep -A 1 ">"$i pdb_seqres.txt ; done > ss.fasta

#### Reduce redundancy via cd-hit

Credits:
Weizhong Li and Adam Godzik. Cd-hit: a fast program for clustering and comparing large sets of protein or nucleotide sequences. Bioinformatics, 2006(22): 1658-1659. 

In [ ]:
# Install cd-hit
!apt-get install cd-hit

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
The following NEW packages will be installed:
  cd-hit
0 upgraded, 1 newly installed, 0 to remove and 42 not upgraded.
Need to get 516 kB of archives.
After this operation, 1,409 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 cd-hit amd64 4.6.8-1 [516 kB]
Fetched 516 kB in 2s (274 kB/s)
Selecting previously unselected package cd-hit.
(Reading database ... 155254 files and directories currently installed.)
Preparing to unpack .../cd-hit_4.6.8-1_amd64.deb ...
Unpacking cd-hit (4.6.8-1) ...
Setting up cd-hit (4.6.8-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [ ]:
# Run cd-hit at 0.95%
!cd-hit -i ss.fasta -o ss_95.fasta -c 0.95

# Check output
!cat ss_95.fasta
!wc ss_95.fasta

Program: CD-HIT, V4.7 (+OpenMP), Jul 01 2017, 08:43:07
Command: cd-hit -i
         /content/drive/MyDrive/Bioinformatics/Lab_bioinfomatics_1_2/ss.fasta
         -o
         /content/drive/MyDrive/Bioinformatics/Lab_bioinfomatics_1_2/ss_95.fasta
         -c 0.95 -n 5

Started: Sun May  8 17:11:04 2022
                            Output                              
----------------------------------------------------------------
total seq: 205
longest and shortest : 100 and 43
Total letters: 12214
Sequences have been sorted

Approximated minimal memory consumption:
Sequence        : 0M
Buffer          : 1 X 10M = 10M
Table           : 1 X 65M = 65M
Miscellaneous   : 0M
Total           : 75M

Table limit with the given memory limit:
Max number of representatives: 4000000
Max number of word counting entries: 90512306

comparing sequences from          0  to        205

      205  finished         21  clusters

Apprixmated maximum memory consumption: 75M
writing new database
writing cluste

#### Multiple structural alignment with PDBeFold

Generate ID:chain txt file for aligning with the MSA tool from PDBeFold. Correct for sequence length.

In [ ]:
# Generate ID:chain txt file
! grep ">" ss_95.fasta | cut -d " " -f 1 | tr -d ">" | tr "_" ":" > ID_for_MSA.txt

# Substitute sequence with length > 90 with seed sequence 3TGI
! sed 's/4bnr:I/3tgi:I/g' ID_for_MSA.txt > final_MSA_ID.txt

# Check output length
!wc ID_for_MSA.txt
!wc final_MSA_ID.txt

# Check if the seed is in the output file
!grep '3tgi' final_MSA_ID.txt 

1brb:I
1dtx:A
1fak:I
1knt:A
1t8l:B
1yc0:I
1zr0:B
3byb:A
3m7q:B
3wny:A
4bnr:I
4dtg:K
4ntx:B
4u30:W
4u32:X
5nmv:K
5nx1:C
5yw1:I
6gfi:C
6q61:A
6yhy:A


#### The txt file containing the identifiers is now submitted manually to the PDBeFold tool for multiple structural alignment, to retrieve the set of sequences that will be used for training the model.

## Validation set

#### From Uniprot/Swissprot manually download sequences of positive hits, according to the following  advanced search criteria:
- Kunitz domain annotation: PFAM code PF00014
- Reviewed entries
- Not PDB (to avoid overlapping with the training set)



In [ ]:
# Check positives

!grep '>' kunitz_positives.fasta |wc

    336    3408   35234


#### From Uniprot/Swissprot manually download sequences of negative hits, according to the following  advanced search criteria:
- Kunitz domain annotation: PFAM code PF00014
- Sequence length: 40 to 10000 res
- Reviewed entries

In [ ]:
# Check negatives

!grep '>' kunitz_negatives.fasta |wc

 557267 8145603 72209028


#### Split both sets for cross-validation procedure



In [ ]:
# Collect positive and negative hits IDs from .fasta files
!grep ">" kunitz_positives.fasta | cut -d "|" -f 2 > kunitz_positive_ID.txt
!grep ">" kunitz_negatives.fasta | cut -d "|" -f 2 > kunitz_negative_ID.txt

# Shuffle the sets
!sort -R kunitz_positive_ID.txt > shuffle_positive_ID.txt
!sort -R kunitz_negative_ID.txt > shuffle_negative_ID.txt


In [ ]:
# Split the sets

# Positive
!head -n 168  shuffle_positive_ID.txt > positive_set1.txt 
!tail -n 168  shuffle_positive_ID.txt > positive_set2.txt

# Negative
!head -n 278633  shuffle_negative_ID.txt >  negative_set1.txt 
!tail -n 278634  shuffle_negative_ID.txt >  negative_set2.txt

# Check for overlapping elements
!comm -12 <(sort  positive_set1.txt) <(sort  positive_set2.txt) |wc
!comm -12 <(sort  negative_set1.txt) <(sort  negative_set2.txt) |wc

In [ ]:
# Extract sequences from validation set fasta file, 
# according to the shuffled and split sets of identifiers

# select_fasta.py is found in the customized_scripts folder of the repository

# Positive set 1
!python select_fasta.py positive_set1.txt kunitz_positives.fasta > positive_set1.fasta

# Positive set 2
!python select_fasta.py positive_set2.txt kunitz_positives.fasta > positive_set2.fasta

# Negative set 1
!python select_fasta.py negative_set1.txt kunitz_negatives.fasta > negative_set1.fasta

# Negative set 2
!python select_fasta.py negative_set2.txt kunitz_negatives.fasta > negative_set2.fasta

In [ ]:
# Check output
!wc positive_set1.txt
!wc positive_set1.fasta

# **HMM model construction**

In [ ]:
# Install HMMER
!apt-get install hmmer

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
The following additional packages will be installed:
  libdivsufsort3
Suggested packages:
  hmmer-doc
The following NEW packages will be installed:
  hmmer libdivsufsort3
0 upgraded, 2 newly installed, 0 to remove and 42 not upgraded.
Need to get 1,164 kB of archives.
After this operation, 11.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libdivsufsort3 amd64 2.0.1-3 [44.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 hmmer amd64 3.1b2+dfsg-5ubuntu1 [1,119 kB]
Fetched 1,164 kB in 1s (1,402 kB/s)
Selecting previously unselected package libdivsufsort3:amd64.
(Reading database ... 155203 files and directories currently installed.)
Preparing to unpack .../lib

In [ ]:
# Build and train the model
!hmmbuild kunitz.hmm kunitz_MSA.txt

# hmmbuild :: profile HMM construction from multiple sequence alignments
# HMMER 3.1b2 (February 2015); http://hmmer.org/
# Copyright (C) 2015 Howard Hughes Medical Institute.
# Freely distributed under the GNU General Public License (GPLv3).
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# input alignment file:             kunitz_MSA.txt
# output HMM file:                  kunitz.hmm
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

# idx name                  nseq  alen  mlen eff_nseq re/pos description
#---- -------------------- ----- ----- ----- -------- ------ -----------
1     kunitz_MSA              21    69    58     2.36  0.961 

# CPU time: 0.06u 0.00s 00:00:00.06 Elapsed: 00:00:00.41


In [ ]:
# Check model
!cat kunitz.hmm

# **Model optimization and validation**

#### Predict with hmmsearch
The command is run with the following parameters:

*   --max: turns off all the heuristics for cutting off distantly related proteins
*   --noali: exclude from the output the alignemnts 
*   --tblout: returns the output in tabular form
*   -Z: for normalizing the e-value output 
*   --domZ: for normalizing the domain e-value output


In [ ]:
!hmmsearch --help

In [ ]:
# Positive set 1
!hmmsearch -Z 1 --domZ 1 --max --noali --tblout positive_set1_pred.match kunitz.hmm positive_set1.fasta

# Positive set 2
!hmmsearch -Z 1 --domZ 1 --max --noali --tblout positive_set2_pred.match kunitz.hmm positive_set2.fasta

# Negative set 1
!hmmsearch -Z 1 --domZ 1 --max --noali --tblout negative_set1_pred.match kunitz.hmm negative_set1.fasta

# Negative set 2
!hmmsearch -Z 1 --domZ 1 --max --noali --tblout negative_set2_pred.match kunitz.hmm negative_set2.fasta

# Check output
!cat positive_set1_pred.match

In [ ]:
# Check number of sequences in output
!grep -v "#"  positive_set1_pred.match |wc
!grep -v "#"  positive_set2_pred.match |wc
!grep -v "#"  negative_set1_pred.match |wc
!grep -v "#"  negative_set2_pred.match |wc

    168    3192   24528
    168    3192   24528
 129317 2457023 18880282
 128731 2445889 18794726


#### Classifying entries from the validation sets 
- positive hits are tagged as class 1
- negative hits are tagged as class 0

In [ ]:
# Positive set 1
!grep -v "#"  positive_set1_pred.match |awk '{print $1,$8,1}' >  positive_set1_eval.class

# Positive set 2
!grep -v "#"  positive_set2_pred.match |awk '{print $1,$8,1}' >  positive_set2_eval.class

# Negative set 1
!grep -v "#"  negative_set1_pred.match |awk '{print $1,$8,0}' >  negative_set1_eval.class

# Negative set 2
!grep -v "#"  negative_set2_pred.match |awk '{print $1,$8,0}' >  negative_set2_eval.class

#### To compute performance metrics with customized python script

The program will take in input a file with three columns representing the following data

1.   Protein ID
2.   The lowest e-value associated to each protein
3.   The class (0: NotBPTI/Kunitz 1: BPTI/Kunitz)

##### Follows the cross-validation optimization procedure performed looping over a list of e-value thresholds with a logarithmic interval. 
It is performed here a 2-fold cross-validation test: thus splitting positives and negatives in 2 subsets, optimizing the classification threshold on one subset and testing the performance on the other subset.

In [ ]:
# Merge positive and negative sets
!cat positive_set1_eval.class negative_set1_eval.class > set1_eval.class
!cat positive_set2_eval.class negative_set2_eval.class > set2_eval.class

In [ ]:
# SET 1 - select eval with highest MCC
!for i in 1e-3 1e-4 1e-5 1e-6 1e-7 1e-8 1e-9 1e-10; do python classification_metrics.py set1_eval.class $i ; done

# Output performance table
!for i in 1e-3 1e-4 1e-5 1e-6 1e-7 1e-8 1e-9 1e-10; do python classification_metrics.py set1_eval.class $i ; done > optimization_table_1.txt

TH: 0.001 Accuracy: 0.9994053365254663 MCC: 0.8275409826704369 TN: 129240.0 FN: 0.0 FP: 77.0 TP: 168.0
TH: 0.0001 Accuracy: 0.9999073251727999 MCC: 0.9654647948127555 TN: 129305.0 FN: 0.0 FP: 12.0 TP: 168.0
TH: 1e-05 Accuracy: 0.9999691083909333 MCC: 0.9876623271725565 TN: 129313.0 FN: 0.0 FP: 4.0 TP: 168.0
TH: 1e-06 Accuracy: 0.9999768312932 MCC: 0.990546023905422 TN: 129314.0 FN: 0.0 FP: 3.0 TP: 168.0
TH: 1e-07 Accuracy: 0.9999691083909333 MCC: 0.9875415572882695 TN: 129314.0 FN: 1.0 FP: 3.0 TP: 167.0
TH: 1e-08 Accuracy: 0.9999768312932 MCC: 0.9904590193741222 TN: 129315.0 FN: 1.0 FP: 2.0 TP: 167.0
TH: 1e-09 Accuracy: 0.9999691083909333 MCC: 0.987453825458062 TN: 129315.0 FN: 2.0 FP: 2.0 TP: 166.0
TH: 1e-10 Accuracy: 0.9999613854886666 MCC: 0.9844394628687669 TN: 129315.0 FN: 3.0 FP: 2.0 TP: 165.0


In [ ]:
# SET 2 - select eval with highest MCC
!for i in 1e-3 1e-4 1e-5 1e-6 1e-7 1e-8 1e-9 1e-10; do python classification_metrics.py set2_eval.class $i ; done

# Output performance table
!for i in 1e-3 1e-4 1e-5 1e-6 1e-7 1e-8 1e-9 1e-10; do python classification_metrics.py set2_eval.class $i ; done > optimization_table_2.txt

TH: 0.001 Accuracy: 0.9994026330693023 MCC: 0.8275385359741207 TN: 128654.0 FN: 0.0 FP: 77.0 TP: 168.0
TH: 0.0001 Accuracy: 0.9998836298186953 MCC: 0.9575156046987876 TN: 128716.0 FN: 0.0 FP: 15.0 TP: 168.0
TH: 1e-05 Accuracy: 0.999968967951652 MCC: 0.9876594103650171 TN: 128727.0 FN: 0.0 FP: 4.0 TP: 168.0
TH: 1e-06 Accuracy: 0.999968967951652 MCC: 0.9875386402131545 TN: 128728.0 FN: 1.0 FP: 3.0 TP: 167.0
TH: 1e-07 Accuracy: 0.9999767259637391 MCC: 0.9904560939352903 TN: 128729.0 FN: 1.0 FP: 2.0 TP: 167.0
TH: 1e-08 Accuracy: 0.9999767259637391 MCC: 0.9904560939352903 TN: 128729.0 FN: 1.0 FP: 2.0 TP: 167.0
TH: 1e-09 Accuracy: 0.9999767259637391 MCC: 0.9904560939352903 TN: 128729.0 FN: 1.0 FP: 2.0 TP: 167.0
TH: 1e-10 Accuracy: 0.9999767259637391 MCC: 0.9904560939352903 TN: 128729.0 FN: 1.0 FP: 2.0 TP: 167.0


Given the following performance on set 1:
- TH: 1e-08 Accuracy: 0.999977 MCC: 0.990456 
- TN: 128729.0 FN: 1.0 FP: 2.0 TP: 167.0

And the following performance on set 2:
- TH: 1e-08 Accuracy: 0.999977 MCC: 0.990459
- TN: 129315.0 FN: 1.0 FP: 2.0 TP: 167.0

We can conclude that an e-value threshold of 1e-08 gives the most optimal performance. 



#### Compare the predicted positives and negatives with the annotated positives and negatives. 

Investigate the reason behind false positives and negatives. 

In [ ]:
!python classification_metrics.py <(cat set1_eval.class set2_eval.class) 1e-8

TH: 1e-08 Accuracy: 0.9999767787479101 MCC: 0.9904575599732265 TN: 258044.0 FN: 2.0 FP: 4.0 TP: 334.0


In [ ]:
# False negatives
!awk '{if ($2 > 1e-08 && $3==1) print $1}' <(cat set1_eval.class set2_eval.class)

O62247
D3GGZ8


In [ ]:
# False positives
!awk '{if ($2 < 1e-08 && $3==0) print $1}' <(cat set1_eval.class set2_eval.class)

P0DV05
P0DV03
P0DV04
P0DV06


#### Brief discussion on false results

O62247, D3GGZ8 are predicted as negatives, but not annotated as such: the annotation of D3GGZ8 is dependent on the other false negative, O62247, used as a seed for it. O62247 is annotated as positive since it appears to have serine protease activity in vitro, but it is uncertain if this activity is genuine, as bli-5 lacks all the catalytic features of serine proteases. It can be considered a borderline case, until further investigation gives a clearer profile to this protein.
For a more detail discussion, you're invited to check (1).


P0DV05, P0DV03, P0DV04, P0DV06 are all isoforms of the same protein, whose sequence actually contains a Kunitz domain as per PROSITE annotation (PRU00031), so it is rightfully predicted as positive. It is not classified as such because the relative UniprotKB entry does not present the PFAM identifier PF00014, used for the construction of the datasets.
For a more detail discussion, you're invited to check (2).

Despite the false results, the performance with a threshold equal to 1e-08 remains valuable, with a Matthew Correlation Coefficient of 0.990457, and an accuracy of 0.999976.

Related references:
1. Stepek G, McCormack G, Page AP. The kunitz domain protein BLI-5 plays a functionally conserved role in cuticle formation in a diverse range of nematodes. Mol Biochem Parasitol. 2010 Jan;169(1):1-11. doi: 10.1016/j.molbiopara.2009.08.005. Epub 2009 Aug 27. PMID: 19716386.
2. Sintsova, O.V., Pislyagin, E.A., Gladkikh, I.N. et al. Kunitz-type peptides of the sea anemone Heteractis crispa: Potential anti-inflammatory compounds. Russ J Bioorg Chem 43, 91–97 (2017). doi: 10.1134/S1068162016060121